In [1]:
!pip install sqlalchemy
%load_ext sql
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
%sql sqlite:///ecommerce_analysis.db

In [3]:
%%sql
CREATE TABLE IF NOT EXISTS order_items (
    order_item_id INT PRIMARY KEY,
    created_at TIMESTAMP,
    order_id INT,
    product_id INT,
    is_primary_item BOOLEAN,
    price_usd DECIMAL(10, 2),
    cogs_usd DECIMAL(10, 2)
);

CREATE TABLE IF NOT EXISTS orders (
    order_id INT PRIMARY KEY,
    created_at TIMESTAMP,
    website_session_id INT,
    user_id INT,
    primary_product_id INT,
    items_purchased INT,
    price_usd DECIMAL(10, 2),
    cogs_usd DECIMAL(10, 2)
);

CREATE TABLE IF NOT EXISTS products (
    product_id INT PRIMARY KEY,
    created_at TIMESTAMP,
    product_name VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS website_sessions (
    website_session_id INT PRIMARY KEY,
    created_at TIMESTAMP,
    user_id INT,
    is_repeat_session BOOLEAN,
    utm_source VARCHAR(255),
    utm_campaign VARCHAR(255),
    utm_content VARCHAR(255),
    device_type VARCHAR(255),
    http_referer VARCHAR(255)
);

 * sqlite:///ecommerce_analysis.db
Done.
Done.
Done.
Done.


[]

In [8]:
# Introduce data
order_items_df = pd.read_csv('order_items.csv', delimiter=';')
orders_df = pd.read_csv('orders.csv', delimiter=';')
products_df = pd.read_csv('products.csv', delimiter=',', quotechar='"')
website_sessions_df = pd.read_csv('website_sessions.csv', delimiter=';')

In [9]:
# Date to DateTime format
order_items_df['created_at'] = pd.to_datetime(order_items_df['created_at'], format='%d/%m/%Y %H:%M')
orders_df['created_at'] = pd.to_datetime(orders_df['created_at'], format='%d/%m/%Y %H:%M')
products_df['created_at'] = pd.to_datetime(products_df['created_at'])
website_sessions_df['created_at'] = pd.to_datetime(website_sessions_df['created_at'], format='%d/%m/%Y %H:%M')

In [10]:
# Clear data from tables

#with engine.connect() as conn:
#conn.execute(text("DELETE FROM order_items"))
#conn.execute(text("DELETE FROM orders"))
#conn.execute(text("DELETE FROM products"))
#conn.execute(text("DELETE FROM website_sessions"))
#conn.commit()

In [11]:
%%sql
SELECT * FROM order_items LIMIT 5;

 * sqlite:///ecommerce_analysis.db
Done.


order_item_id,created_at,order_id,product_id,is_primary_item,price_usd,cogs_usd
1,2012-03-19 10:42:00.000000,1,1,1,49.99,19.49
2,2012-03-19 19:27:00.000000,2,1,1,49.99,19.49
3,2012-03-20 06:44:00.000000,3,1,1,49.99,19.49
4,2012-03-20 09:41:00.000000,4,1,1,49.99,19.49
5,2012-03-20 11:28:00.000000,5,1,1,49.99,19.49


In [12]:
%%sql
SELECT * FROM orders LIMIT 5;

 * sqlite:///ecommerce_analysis.db
Done.


order_id,created_at,website_session_id,user_id,primary_product_id,items_purchased,price_usd,cogs_usd
1,2012-03-19 10:42:00.000000,20,20,1,1,49.99,19.49
2,2012-03-19 19:27:00.000000,104,104,1,1,49.99,19.49
3,2012-03-20 06:44:00.000000,147,147,1,1,49.99,19.49
4,2012-03-20 09:41:00.000000,160,160,1,1,49.99,19.49
5,2012-03-20 11:28:00.000000,177,177,1,1,49.99,19.49


In [13]:
%%sql
SELECT * FROM products LIMIT 5;

 * sqlite:///ecommerce_analysis.db
Done.


product_id,created_at,product_name
"1,""2012-03-19 08:00:00"",""Barefoot Running""",None,None
"2,""2013-01-06 13:00:00"",""Barefoot Casual""",None,None
"3,""2013-12-12 09:00:00"",""Barefoot Senderismo""",None,None


In [14]:
%%sql
SELECT * FROM website_sessions LIMIT 5;

 * sqlite:///ecommerce_analysis.db
Done.


website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
1,2012-03-19 08:04:00.000000,1,0,gsearch,brand awareness,g_ad_1,mobile,https://www.gsearch.com
2,2012-03-19 08:16:00.000000,2,0,gsearch,brand awareness,g_ad_1,desktop,https://www.gsearch.com
3,2012-03-19 08:26:00.000000,3,0,gsearch,brand awareness,g_ad_1,desktop,https://www.gsearch.com
4,2012-03-19 08:37:00.000000,4,0,gsearch,brand awareness,g_ad_1,desktop,https://www.gsearch.com
5,2012-03-19 09:00:00.000000,5,0,gsearch,brand awareness,g_ad_1,mobile,https://www.gsearch.com


In [16]:
engine = create_engine('sqlite:///ecommerce_analysis.db', echo=False)
conn = engine.connect()

## Sales Analysis

In [17]:
# What are the gross sales and absolute margins by year and month?

query = text("""
SELECT 
    strftime('%Y', created_at) AS year, 
    strftime('%m', created_at) AS month, 
    SUM(price_usd * items_purchased) AS gross_sales, 
    SUM((price_usd - cogs_usd) * items_purchased) AS absolute_margin
FROM orders
GROUP BY year, month;
""")

df1 = pd.read_sql_query(query, conn)   
df1

,year,month,gross_sales,absolute_margin
0,2012,03,2999.40,1830.00
1,2012,04,4949.01,3019.50
2,2012,05,5398.92,3294.00
3,2012,06,6998.60,4270.00
4,2012,07,8448.31,5154.50
5,2012,08,11397.72,6954.00
6,2012,09,14347.13,8753.50
7,2012,10,18546.29,11315.50
8,2012,11,32653.82,20256.12
9,2012,12,35414.94,23523.94


In [18]:
# What are the top 10 average gross sales by month and year?

query = text("""
SELECT 
    strftime('%Y', created_at) AS year, 
    strftime('%m', created_at) AS month, 
    AVG(price_usd * items_purchased) AS avg_gross_sales
FROM orders
GROUP BY year, month
ORDER BY avg_gross_sales DESC
LIMIT 10;
""")

df2 = pd.read_sql_query(query, conn)   
df2

,year,month,avg_gross_sales
0,2012,12,69.990000
1,2013,01,68.784872
2,2013,02,68.390000
3,2012,11,52.837896
4,2012,10,49.990000
5,2012,03,49.990000
6,2012,04,49.990000
7,2012,05,49.990000
8,2012,06,49.990000
9,2012,07,49.990000


In [19]:
# Which product has the highest sales in monetary terms (gross sales)?

query = text("""
SELECT 
    p.product_name, 
    SUM(oi.price_usd * oi.is_primary_item) AS gross_sales
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_name
ORDER BY gross_sales DESC
LIMIT 1;
""")

df3 = pd.read_sql_query(query, conn)
df3

,product_name,gross_sales


In [27]:
# Which product has the highest profit margin?

query = text("""
SELECT 
    p.product_name, 
    SUM((oi.price_usd - oi.cogs_usd) * oi.is_primary_item) AS total_margin
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_margin DESC
LIMIT 1;
""")

df4 = pd.read_sql_query(query, conn)
df4

,product_name,total_margin


In [21]:
# Can we determine the launch date of each product?

query = text("""
SELECT 
    p.product_name, 
    MIN(o.created_at) AS launch_date
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_name;
""")

df5 = pd.read_sql_query(query, conn)
df5


,product_name,launch_date


In [28]:
# Calculate the gross sales by year and the numeric and percentage margin of each product, ordered by product.

query = text("""
SELECT 
    strftime('%Y', o.created_at) AS year, 
    p.product_name, 
    SUM(oi.price_usd * oi.is_primary_item) AS gross_sales, 
    SUM((oi.price_usd - oi.cogs_usd) * oi.is_primary_item) AS absolute_margin,
    (SUM((oi.price_usd - oi.cogs_usd) * oi.is_primary_item) / SUM(oi.price_usd * oi.is_primary_item)) * 100 AS percentage_margin
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY year, p.product_name
ORDER BY p.product_name;
""")

df6 = pd.read_sql_query(query, conn)
df6


,year,product_name,gross_sales,absolute_margin,percentage_margin


In [23]:
# What are the top 3 months with the highest gross sales?

query = text("""
SELECT 
    strftime('%Y-%m', created_at) AS year_month, 
    SUM(price_usd * items_purchased) AS gross_sales
FROM orders
GROUP BY year_month
ORDER BY gross_sales DESC
LIMIT 3;
""")

df7 = pd.read_sql_query(query, conn)
df7

,year_month,gross_sales
0,2012-12,35414.94
1,2012-11,32653.82
2,2013-01,26826.10


## Web Traffic Analysis

In [37]:
# Which ads or content have attracted the most sessions?

query = text("""
SELECT 
    utm_content, 
    COUNT(*) AS total_sessions
FROM website_sessions
GROUP BY utm_content
ORDER BY total_sessions DESC
LIMIT 10;
""")

df8 = pd.read_sql_query(query, conn)
df8

,utm_content,total_sessions
0,g_ad_1,49978
1,b_ad_1,9208
2,None,7248
3,g_ad_2,2876
4,b_ad_2,690


In [36]:
# Are sessions the same as users? What is the number of unique users?

query = text("""
SELECT 
    COUNT(DISTINCT user_id) AS unique_users, 
    COUNT(*) AS total_sessions
FROM website_sessions;
""")

df9 = pd.read_sql_query(query, conn)
df9

,unique_users,total_sessions
0,62224,70000


In [35]:
# What is the number of users and sessions by source?

query = text("""
SELECT 
    utm_source, 
    COUNT(DISTINCT user_id) AS unique_users, 
    COUNT(*) AS total_sessions
FROM website_sessions
GROUP BY utm_source
ORDER BY total_sessions DESC;
""")

df10 = pd.read_sql_query(query, conn)
df10

,utm_source,unique_users,total_sessions
0,gsearch,51118,52854
1,bsearch,9808,9898
2,None,5899,7248


In [34]:
# Which sources have generated the most sales?

query = text("""
SELECT 
    ws.utm_source, 
    SUM(oi.price_usd * oi.is_primary_item) AS gross_sales
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN website_sessions ws ON o.user_id = ws.user_id  -- Ajustar aquí si user_id es la columna común
GROUP BY ws.utm_source
ORDER BY gross_sales DESC
LIMIT 10;
""")

df11 = pd.read_sql_query(query, conn)
df11

,utm_source,gross_sales
0,gsearch,153182.97
1,None,50391.33
2,bsearch,35363.85


In [33]:
# Which months have attracted the most traffic?

query = text("""
SELECT 
    strftime('%Y-%m', created_at) AS year_month, 
    COUNT(*) AS total_sessions
FROM website_sessions
GROUP BY year_month
ORDER BY total_sessions DESC
LIMIT 3;
""")

df12 = pd.read_sql_query(query, conn)
df12

,year_month,total_sessions
0,2012-11,14011
1,2012-12,10072
2,2012-10,8183


In [31]:
# For the month with the highest traffic, what is the number of sessions from mobile and desktop?

query_most_traffic_month = text("""
SELECT 
    strftime('%Y-%m', created_at) AS year_month, 
    COUNT(*) AS total_sessions
FROM website_sessions
GROUP BY year_month
ORDER BY total_sessions DESC
LIMIT 1;
""")

df_most_traffic_month = pd.read_sql_query(query_most_traffic_month, conn)
most_traffic_month = df_most_traffic_month.iloc[0]['year_month']
most_traffic_month

'2012-11'

In [32]:
query = text(f"""
SELECT 
    device_type, 
    COUNT(*) AS total_sessions
FROM website_sessions
WHERE strftime('%Y-%m', created_at) = '{most_traffic_month}'
GROUP BY device_type;
""")

df13 = pd.read_sql_query(query, conn)
df13

,device_type,total_sessions
0,desktop,10871
1,mobile,3140


In [29]:
# Which campaigns have generated the most profit margin by product?

query = text("""
SELECT 
    ws.utm_campaign, 
    p.product_name, 
    SUM((oi.price_usd - oi.cogs_usd) * oi.is_primary_item) AS total_margin
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN products p ON oi.product_id = p.product_id
JOIN website_sessions ws ON o.user_id = ws.user_id
GROUP BY ws.utm_campaign, p.product_name
ORDER BY total_margin DESC
LIMIT 10;
""")

df14 = pd.read_sql_query(query, conn)
df14

,utm_campaign,product_name,total_margin
